# **Configuración Maestra (Instalación + Vinculación a Drive)**

In [1]:
import os
import shutil
from google.colab import drive

# 1. Conectar Google Drive
drive.mount('/content/drive')

# 2. Definir Rutas
base_path = "/content"
comfy_path = f"{base_path}/ComfyUI"
# Esta será la carpeta raíz en tu Drive para guardar tus cosas
GDRIVE_BASE = '/content/drive/MyDrive/ComfyUI_Data'

# 3. Instalar Herramientas de Aceleración (Aria2)
!apt-get -y install -qq aria2

# 4. Instalar ComfyUI en disco local de Colab (Efímero - Máxima Velocidad)
if not os.path.exists(comfy_path):
    print("⚡ Instalando ComfyUI en el disco local de Colab...")
    %cd {base_path}
    !git clone https://github.com/comfyanonymous/ComfyUI
    %cd {comfy_path}
    !pip install xformers!=0.0.18 -r requirements.txt --extra-index-url https://download.pytorch.org/whl/cu121

    # Instalar Manager y soporte GGUF (Necesario para Flux)
    %cd custom_nodes
    !git clone https://github.com/ltdrdata/ComfyUI-Manager
    !git clone https://github.com/city96/ComfyUI-GGUF
    !pip install -r ComfyUI-GGUF/requirements.txt
    %cd ..
else:
    print("✅ ComfyUI ya está instalado.")

# 5. CONFIGURACIÓN DE ENLACES (La parte inteligente)
# Esto conecta las carpetas de Colab a tu Drive solo para lo importante.

LINKS = {
    'output': f'{GDRIVE_BASE}/output',      # Tus imágenes generadas
    'input':  f'{GDRIVE_BASE}/input',       # Imágenes que subas para editar
    'user':   f'{GDRIVE_BASE}/user'         # Tus Workflows guardados y config
}

# Asegurar que la base en Drive exista
if not os.path.exists(GDRIVE_BASE):
    os.makedirs(GDRIVE_BASE, exist_ok=True)

print("🔄 Configurando enlaces simbólicos inteligentes...")

for name, target in LINKS.items():
    source = os.path.join(comfy_path, name)

    # A. Asegurar que la carpeta destino exista en Drive
    if not os.path.exists(target):
        os.makedirs(target, exist_ok=True)
        print(f"   📂 Creada carpeta en Drive: {target}")

    # B. Eliminar la carpeta local actual (para reemplazarla por el enlace)
    if os.path.exists(source):
        if os.path.islink(source):
            os.unlink(source) # Borrar si ya era un enlace
        elif os.path.isdir(source):
            shutil.rmtree(source) # Borrar si era una carpeta normal

    # C. Crear el enlace simbólico (Puente)
    os.symlink(target, source)
    print(f"   🔗 Vinculado: {name} -> Drive")

print("\n✅ ¡Listo! Sistema Híbrido Activo.")

Mounted at /content/drive
Selecting previously unselected package libc-ares2:amd64.
(Reading database ... 121713 files and directories currently installed.)
Preparing to unpack .../libc-ares2_1.18.1-1ubuntu0.22.04.3_amd64.deb ...
Unpacking libc-ares2:amd64 (1.18.1-1ubuntu0.22.04.3) ...
Selecting previously unselected package libaria2-0:amd64.
Preparing to unpack .../libaria2-0_1.36.0-1_amd64.deb ...
Unpacking libaria2-0:amd64 (1.36.0-1) ...
Selecting previously unselected package aria2.
Preparing to unpack .../aria2_1.36.0-1_amd64.deb ...
Unpacking aria2 (1.36.0-1) ...
Setting up libc-ares2:amd64 (1.18.1-1ubuntu0.22.04.3) ...
Setting up libaria2-0:amd64 (1.36.0-1) ...
Setting up aria2 (1.36.0-1) ...
Processing triggers for man-db (2.10.2-1) ...
Processing triggers for libc-bin (2.35-0ubuntu3.8) ...
/sbin/ldconfig.real: /usr/local/lib/libur_adapter_level_zero.so.0 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libtbbbind.so.3 is not a symbolic link

/sbin/ldconfig.real: /us

# **Gestor de Descargas (Modelos Flux + GGUF)**

In [ ]:
import os

# ==============================================================================
# 📋 LISTA MAESTRA DE MODELOS (ENLACES ARREGLADOS)
# ==============================================================================

DOWNLOAD_LIST = [
    # --- FLUX BASE ---
    ("https://huggingface.co/Comfy-Org/flux1-schnell/resolve/main/flux1-schnell-fp8.safetensors?download=true", "flux1-schnell-fp8.safetensors", "checkpoints"),
    ("https://huggingface.co/Comfy-Org/flux1-dev/resolve/main/flux1-dev-fp8.safetensors?download=true", " flux1-dev-fp8.safetensors", "checkpoints"),
]

# ==============================================================================
# ⚙️ MOTOR DE DESCARGA
# ==============================================================================
print("🚀 Iniciando Gestor de Descargas...")

base_path = "/content/ComfyUI/models"

# HE AGREGADO LAS CARPETAS QUE FALTABAN AQUÍ ABAJO 👇
folders = {
    "unet": f"{base_path}/unet",
    "clip": f"{base_path}/clip",
    "vae": f"{base_path}/vae",
    "lora": f"{base_path}/loras",
    "loras": f"{base_path}/loras",  # Alias por si escribes 'loras' en plural
    "checkpoint": f"{base_path}/checkpoints", # Alias por si escribes 'checkpoints' en plural
    "checkpoints": f"{base_path}/checkpoints",
    "controlnet": f"{base_path}/controlnet",
    "clip_vision": f"{base_path}/clip_vision",        # NUEVO
    "diffusion_models": f"{base_path}/diffusion_models", # NUEVO (Para Wan/Chrono)
    "text_encoders": f"{base_path}/text_encoders"     # NUEVO (ComfyUI usa esta carpeta a veces)
}

# Crear todas las carpetas
for path in folders.values():
    os.makedirs(path, exist_ok=True)

for url, filename, type_key in DOWNLOAD_LIST:
    # Ahora sí encontrará las claves nuevas
    if type_key not in folders:
        print(f"⚠️ Error: No existe la categoría '{type_key}' en el diccionario de carpetas.")
        continue

    destination = folders[type_key]
    file_path = os.path.join(destination, filename)

    if os.path.exists(file_path):
        print(f"✅ Ya existe: {filename}")
    else:
        print(f"⬇️ Descargando: {filename} en [{type_key}]...")
        # Usamos user-agent para evitar bloqueos en archivos grandes
        !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M --user-agent="Mozilla/5.0" "{url}" -d "{destination}" -o "{filename}"

print("\n✨ ¡Descargas corregidas y completadas!")

# **Ejecución (Pinggy + ComfyUI)**

In [ ]:
import os

print("💾 Creando archivo de intercambio (SWAP) para aumentar la RAM del sistema...")

# 1. Verificar si ya existe swap
if os.path.exists('/swapfile'):
    print("✅ El archivo Swap ya existe.")
else:
    # 2. Crear un archivo de 8GB en el disco
    # Usamos 'fallocate' que es más rápido, o dd si falla
    !fallocate -l 8G /swapfile
    !chmod 600 /swapfile
    !mkswap /swapfile
    !swapon /swapfile
    print("✅ Swap de 8GB creado y activado.")

# 3. Verificar memoria total ahora
print("\n📊 Memoria Total Disponible (RAM + SWAP):")
!free -h

In [ ]:
import subprocess
import threading
import time
import socket
import os

# 1. LIMPIEZA
!pkill -9 -f "python main.py"
!pkill -9 -f cloudflared
!pkill -9 -f ngrok
!pkill -9 ssh
time.sleep(1)

%cd /content/ComfyUI

# 2. PINGGY
def pinggy_tunnel(port):
    print(f"Intentando abrir túnel en el puerto {port}...")
    cmd = f"ssh -p 443 -R0:localhost:{port} -o StrictHostKeyChecking=no -o ServerAliveInterval=30 a.pinggy.io"
    process = subprocess.Popen(cmd.split(), stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    for line in process.stdout:
        l = line.decode().strip()
        if "pinggy.link" in l:
            print("\n" + "★"*50)
            print("✅ ¡TÚNEL ESTABLE GENERADO!")
            print(f"👉 ENTRA AQUÍ: {l}")
            print("★"*50 + "\n")

threading.Thread(target=pinggy_tunnel, daemon=True, args=(8188,)).start()

print("Iniciando ComfyUI... (Modo Supervivencia: LowVRAM + Swap)")

# Usamos --lowvram. No usamos --cpu porque sería eterno.
!python main.py --dont-print-server --listen 0.0.0.0 --port 8188 --enable-cors-header "*" --preview-method auto --lowvram